In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

/home/norrman/anaconda3/envs/thesis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
XGLM_tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-564M", padding_side='left')
XGLM_model = AutoModelForCausalLM.from_pretrained("facebook/xglm-564M")

XGLM_pipe = pipeline("text-generation", model="facebook/xglm-564M")

In [41]:
def make_prompts(data):
    prefix = {'ja': '日本語で答えてください。',
              'en': 'Please answer the question in English.',
              'de': 'Bitte beantworten Sie die Frage auf Deutsch.',
              'fi': 'Vastaa kysymykseen suomeksi.'}
    return [prefix[item['lang']] + f" C: {item['context']} Q: {item['question']} A: " for item in data]

In [42]:
data = [{'lang': 'ja',
        'context': '陽向と陽翔は弁護士ではないですが、誠と清はそうです。',
         'question': '何方が弁護士ですか。',
         'answer': '誠と清です。'},
         {'lang': 'en',
        'context': 'Emma and Monica are not doctors, but Mark and Andrew are.',
         'question': 'Who are doctors?',
         'answer': 'Mark and Andrew.'},
        {'lang': 'de',
        'context': 'Anna und Ulrike sind keine Ingenieurinnen, aber Robert und Hans sind es.',
         'question': 'Wer sind Ingenieure?',
         'answer': 'Robert und Hans.'},
         {'lang': 'fi',
        'context': 'Hilppa ja Henriikka eivät ole poliiseja mutta Kaija ja Juuli ovat.',
         'question': 'Ketkä ovat poliisit?',
         'answer': 'Kaija ja Juuli.'}]


In [43]:
XGLM_pipe(make_prompts(data), return_full_text=True, max_new_tokens=15)

[[{'generated_text': '日本語で答えてください。 C: 陽向と陽翔は弁護士ではないですが、誠と清はそうです。 Q: 何方が弁護士ですか。 A:  誠と清は、弁護士です。 Q: 誠と'}],
 [{'generated_text': 'Please answer the question in English. C: Emma and Monica are not doctors, but Mark and Andrew are. Q: Who are doctors? A:  Mark and Andrew are doctors. Q: What is the difference between a doctor'}],
 [{'generated_text': 'Bitte beantworten Sie die Frage auf Deutsch. C: Anna und Ulrike sind keine Ingenieurinnen, aber Robert und Hans sind es. Q: Wer sind Ingenieure? A:  Ingenieure sind diejenigen, die sich für die technische Entwicklung und die'}],
 [{'generated_text': 'Vastaa kysymykseen suomeksi. C: Hilppa ja Henriikka eivät ole poliiseja mutta Kaija ja Juuli ovat. Q: Ketkä ovat poliisit? A:  Kaija ja Juuli. Q: Ketkä ovat poliiseja?'}]]

In [44]:
inputs = XGLM_tokenizer(make_prompts(data), return_tensors='pt', padding=True)
outputs = XGLM_model.generate(**inputs, 
                              max_new_tokens=15,
                              num_beams=1)

XGLM_tokenizer.batch_decode(outputs, skip_special_tokens=True)

['日本語で答えてください。 C: 陽向と陽翔は弁護士ではないですが、誠と清はそうです。 Q: 何方が弁護士ですか。 A: 誠と清は、弁護士です。 Q: 誠と',
 'Please answer the question in English. C: Emma and Monica are not doctors, but Mark and Andrew are. Q: Who are doctors? A: Mark and Andrew are doctors. Q: What is the difference between a doctor',
 'Bitte beantworten Sie die Frage auf Deutsch. C: Anna und Ulrike sind keine Ingenieurinnen, aber Robert und Hans sind es. Q: Wer sind Ingenieure? A: Die Ingenieure sind diejenigen, die sich für die technische Entwicklung der',
 'Vastaa kysymykseen suomeksi. C: Hilppa ja Henriikka eivät ole poliiseja mutta Kaija ja Juuli ovat. Q: Ketkä ovat poliisit? A: Kaija ja Juuli. Q: Ketkä ovat poliiseja?']